In [14]:
import json, urllib

In [15]:
with urllib.request.urlopen("https://raw.githubusercontent.com/hebcal/hebcal-leyning/master/src/aliyot.json") as url:
    aliyot = json.loads(url.read().decode())
with urllib.request.urlopen("https://www.sefaria.org/api/texts/Exodus?vhe=Miqra_according_to_the_Masorah&context=0&pad=0") as url:
    MAPM_ex = json.loads(url.read().decode())    
with urllib.request.urlopen("https://raw.githubusercontent.com/Sefaria/Sefaria-Export/master/json/Tanakh/Torah/Exodus/English/The%20Holy%20Scriptures%20A%20New%20Translation%20JPS%201917.json") as url:
    JPS_ex = json.loads(url.read().decode())
with urllib.request.urlopen("https://raw.githubusercontent.com/Sefaria/Sefaria-Export/master/json/Tanakh/Targum/Onkelos/Torah/Onkelos%20Exodus/Hebrew/Targum%20Onkelos%2C%20vocalized%20according%20to%20the%20Yemenite%20Taj.json") as url:
    targ_ex = json.loads(url.read().decode())
with urllib.request.urlopen("https://raw.githubusercontent.com/Sefaria/Sefaria-Export/master/json/Tanakh/Commentary/Rashi/Torah/Rashi%20on%20Exodus/Hebrew/Rashi%20Chumash%2C%20Metsudah%20Publications%2C%202009.json") as url:
    rashi_ex = json.loads(url.read().decode())

In [16]:
def qkparse(string):
    start = string.index('<span class=\"mam-kq">')
    string_before = string[0:start]
    startk = string.index('<span class=\"mam-kq-k\">',start)
    startk = string.index('>',startk)+2
    endk = string.index('</span>',startk)-1
    ktext = string[startk:endk]
    startq = string.index('<span class=\"mam-kq-q\">')
    startq = string.index('>',startq)+2
    endq = string.index('</span>',startq)-1
    qtext=string[startq:endq]
    end = string.index('</span></span>',endq)
    if " " in string[end:]:
        end = string.index(' ',end)
    else:
        end = string.index('׃',end)
    output = '\\qk{'+qtext+'}{'+ktext+'}'
    string_after = string[end:]
    return string_before+output+string_after

In [213]:
def removeformatting(text):
    text = text.replace("{פ}",r"\petucha ")
    text = text.replace("{ס}",r"\setuma ")
    text = text.replace("<span>","")
    text = text.replace("</span>","")
    text = text.replace("<br>","")
    text = text.replace("<span class=\"mam-kq-trivial\">","")
    text = text.replace("<b>׀</b>",r"\legarmeh ")
    text = text.replace("<small>׀</small>",r"\pasek ")
    text = text.replace("<big>",r"\large ")
    text = text.replace("</big>",r"\normalsize ")
    text = text.replace("<sup>*</sup><i class=\"footnote\">(",r"\note{")
    text = text.replace(')</i>',r"}")
    text = text.replace("           "," ")
    if "<" in text:
        print(text)
    return text

In [114]:
def parserashi(content, chap, vs):
    rashi_chap = content["text"][chap-1]
    if len(rashi_chap)<vs:
        return ""
    rashi_list = rashi_chap[vs-1]
    rashi_text = ""
    for comment in rashi_list:
        if comment == "":
            continue
        comment = comment.replace("<b>",r"\rashiDH{")
        comment = comment.replace("</b>",r"}")
        comment = comment.replace("<small>","")
        comment = comment.replace("</small>","")
        comment = comment.replace(r"\"","״")
        comment = comment.replace("\"","״")
        comment = comment.replace(r"'","׳")
        comment = comment.replace(r":","׃")
        comment = comment.replace(r"—","")
        rashi_text += r"\rashi{"+comment+r"}"
    return rashi_text

In [115]:
def parseenglish(text):
    text = text.replace(r"s’",r"s’ ")
    text = text.replace("LORD",r"\lord")
    text = text.replace(r"\lord ",r"\lord\space ")
    text = text.replace("I AM THAT I AM",r"\textsc{I Am That I Am}")
    text = text.replace("I AM",r"\textsc{I Am}")
    text = text.replace("יהוה",r"\englishshem")
    text = text.replace("GOD",r"\textsc{God}")
    return text

In [116]:
def nonalpha_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter.isalpha() == True:
            no_cant_word += letter
        if letter == "-" or letter == " " or letter == "־":
            no_cant_word += " "
    return no_cant_word

In [59]:
def parsealiyot(aliyot,book_num,vs_id):
    aliyot = dict(reversed(list(aliyot.items())))
    combo = None
    for parsha in aliyot.values():
        if parsha["book"] != book_num:
            continue
        elif parsha["fullkriyah"]["1"]["b"] == vs_id and "combined" not in parsha.keys():
            return ["parsha",nonalpha_remover(parsha["hebrew"])]
        elif parsha["fullkriyah"]["7"]["e"] == vs_id and "combined" not in parsha.keys():
            return ["parsha","end"]
        elif "combined" in parsha.keys():
            aliya_number = 1
            for aliya in parsha["fullkriyah"].values():
                if aliya["b"] == vs_id:
                    combo = aliya_number
                aliya_number += 1
        else:
            aliya_number = 1
            for aliya in parsha["fullkriyah"].values():
                if aliya["b"] == vs_id:
                    if combo == None or aliya_number == 8:
                        return ["aliya",aliya_number]
                    else:
                        return ["combo",[aliya_number, combo]]
                aliya_number += 1
            aliya_number = 1
            if "weekday" in parsha.keys():
                for weekday in parsha["weekday"].values():
                    if weekday["b"]==vs_id:
                        return ["weekday",aliya_number]
                    elif vs_id == parsha["weekday"]["3"]["e"] and vs_id != parsha["fullkriyah"]["1"]["e"]:
                        return ["weekday",4]
                    aliya_number += 1
    return

In [132]:
def endofparsha(parsha_dict):
    haftarah_ashk = parsha_dict["haftara"]
    haftarah_text = "The Haftarah "
    if "sephardic" in parsha_dict.keys():
        haftarah_text += "for Ashkenazim is "+haftarah_ashk+", and for Sepharadim is "+ parsha_dict["sephardic"]
    else:
        haftarah_text += " is" +haftarah_ashk
    return haftarah_text

In [133]:
endofparsha(aliyot["Shemot"])

'The Haftarah for Ashkenazim is Isaiah 27:6 - 28:13; 29:22 - 29:23, and for Sepharadim is Jeremiah 1:1 - 2:3'

In [134]:
weekday_aliya_names = [None, None,"לוי","ישראל","ע״כ בחול"]
shabbat_aliya_names = [None,None,"שני","שלישי","רביעי","חמישי","ששי","שביעי","מפטיר"]

In [161]:
taam_tachton_ex = ["","","אָֽנֹכִ֖י יְהֹוָ֣ה אֱלֹהֶ֑יךָ אֲשֶׁ֧ר הוֹצֵאתִ֛יךָ מֵאֶ֥רֶץ מִצְרַ֖יִם מִבֵּ֥ית עֲבָדִֽים׃",
"לֹֽא־יִהְיֶ֥ה לְךָ֛ אֱלֹהִ֥ים אֲחֵרִ֖ים עַל־פָּנָֽי׃",
"לֹֽא־תַעֲשֶׂ֨ה לְךָ֥ פֶ֙סֶל֙ וְכׇל־תְּמוּנָ֔ה אֲשֶׁ֤ר בַּשָּׁמַ֙יִם֙ מִמַּ֔עַל וַֽאֲשֶׁ֥ר בָּאָ֖רֶץ מִתָּ֑חַת וַאֲשֶׁ֥ר בַּמַּ֖יִם מִתַּ֥חַת לָאָֽרֶץ׃",
"לֹֽא־תִשְׁתַּחֲוֶ֥ה לָהֶ֖ם וְלֹ֣א תׇעׇבְדֵ֑ם כִּ֣י אָֽנֹכִ֞י יְהֹוָ֤ה אֱלֹהֶ֙יךָ֙ אֵ֣ל קַנָּ֔א פֹּ֠קֵ֠ד עֲוֺ֨ן אָבֹ֧ת עַל־בָּנִ֛ים עַל־שִׁלֵּשִׁ֥ים וְעַל־רִבֵּעִ֖ים לְשֹׂנְאָֽי׃",
"וְעֹ֥שֶׂה חֶ֖סֶד לַאֲלָפִ֑ים לְאֹהֲבַ֖י וּלְשֹׁמְרֵ֥י מִצְוֺתָֽי׃"+r"\setuma",
"לֹ֥א תִשָּׂ֛א אֶת־שֵֽׁם־יְהֹוָ֥ה אֱלֹהֶ֖יךָ לַשָּׁ֑וְא כִּ֣י לֹ֤א יְנַקֶּה֙ יְהֹוָ֔ה אֵ֛ת אֲשֶׁר־יִשָּׂ֥א אֶת־שְׁמ֖וֹ לַשָּֽׁוְא׃"+r"\petucha",
"זָכ֛וֹר אֶת־י֥וֹם הַשַּׁבָּ֖ת לְקַדְּשֽׁוֹ׃",
"שֵׁ֤שֶׁת יָמִים֙ תַּֽעֲבֹ֔ד וְעָשִׂ֖יתָ כׇּל־מְלַאכְתֶּֽךָ׃",
"וְיוֹם֙ הַשְּׁבִיעִ֔י שַׁבָּ֖ת לַיהֹוָ֣ה אֱלֹהֶ֑יךָ לֹֽא־תַעֲשֶׂ֨ה כׇל־מְלָאכָ֜ה אַתָּ֣ה ׀ וּבִנְךָ֣ וּבִתֶּ֗ךָ עַבְדְּךָ֤ וַאֲמָֽתְךָ֙ וּבְהֶמְתֶּ֔ךָ וְגֵרְךָ֖ אֲשֶׁ֥ר בִּשְׁעָרֶֽיךָ׃",
"כִּ֣י שֵֽׁשֶׁת־יָמִים֩ עָשָׂ֨ה יְהֹוָ֜ה אֶת־הַשָּׁמַ֣יִם וְאֶת־הָאָ֗רֶץ אֶת־הַיָּם֙ וְאֶת־כׇּל־אֲשֶׁר־בָּ֔ם וַיָּ֖נַח בַּיּ֣וֹם הַשְּׁבִיעִ֑י עַל־כֵּ֗ן בֵּרַ֧ךְ יְהֹוָ֛ה אֶת־י֥וֹם הַשַּׁבָּ֖ת וַֽיְקַדְּשֵֽׁהוּ׃"+r"\setuma",
"כַּבֵּ֥ד אֶת־אָבִ֖יךָ וְאֶת־אִמֶּ֑ךָ לְמַ֙עַן֙ יַאֲרִכ֣וּן יָמֶ֔יךָ עַ֚ל הָאֲדָמָ֔ה אֲשֶׁר־יְהֹוָ֥ה אֱלֹהֶ֖יךָ נֹתֵ֥ן לָֽךְ׃"+r"\setuma",
r"לֹ֥א תִרְצָ֖ח \setuma לֹ֣א תִנְאָ֑ף \setuma לֹ֣א תִגְנֹ֔ב לֹֽא־תַעֲנֶ֥ה בְרֵעֲךָ֖ עֵ֥ד שָֽׁקֶר׃ \setuma",
"לֹ֥א תַחְמֹ֖ד בֵּ֣ית רֵעֶ֑ךָ לֹֽא־תַחְמֹ֞ד אֵ֣שֶׁת רֵעֶ֗ךָ וְעַבְדּ֤וֹ וַאֲמָתוֹ֙ וְשׁוֹר֣וֹ וַחֲמֹר֔וֹ וְכֹ֖ל אֲשֶׁ֥ר לְרֵעֶֽךָ׃"+r"\petucha"]

In [181]:
def specialcase(heb, book, chap, verse):
    if book == 2 and chap == 20 and 1 < verse < 15:
        heb = taam_tachton_ex[verse]
    if book == 2 and chap == 15 and verse < 20:
        heb = heb.replace(r"\setuma","")
    return heb

In [203]:
def addnotes(booknum, chapnum, vsnum):
    note_text = None
    if booknum == 2 and chapnum==20 and vsnum==2:
        note_text = r"The Ten Commandments are presented here in Ta'am Ta\d{h}ton. For the Ten Commandments in Ta'am Elyon, see page \pageref{elyon}."
    elif booknum == 2 and chapnum == 15 and vsnum==1:
        note_text = r"For brevity, all notes of breaks in the Song of the Sea are omitted. For the proper formatting for this passage, see page \pageref{shirathayam}."
    return note_text

In [214]:
def make_chumash(heb, targ, eng, rashi, parsha_data, book):
    versecounter = 1
    chapcounter = 1
    output = []
    for chapter in heb["he"]:
        for verse in chapter:
            id_str = str(chapcounter)+":"+str(versecounter)
            verse_he = verse
            while '<span class=\"mam-kq\">' in verse_he:
                verse_he = qkparse(verse_he)
            verse_he = removeformatting(verse_he)
            verse_he = specialcase(verse_he,book,chapcounter,versecounter)
            heb_id = parsealiyot(parsha_data, 2, id_str)
            if heb_id == None:
                if versecounter == 1:
                    versetext_he = r'\Roman{chap}'
                else:
                    versetext_he = r"\arabic{verse}"
            elif heb_id[0] == "parsha" and heb_id[1]!= "end":
                #if chapcounter != 1:
                    #output.append(r"\end{longtable}")
                    #output.append(r"}")
                #add end-of-parsha note here
                output.append(r"\newparsha{"+heb_id[1]+"}")
                versetext_he = r"\aliya{"+heb_id[1]+"}"
                #output.append(r"\parshabody{")
                #if versecounter == 1:
                #    versetext_he = r'\Roman{chap}'
                #else:
                #    versetext_he = r"\arabic{verse}"
            elif heb_id[0] == "weekday":
                aliya_text = weekday_aliya_names[heb_id[1]]
                versetext_he = r"\aliya{"+aliya_text+"}"
            elif heb_id[0] == "combo":
                aliya_text = [shabbat_aliya_names[heb_id[1][1]],shabbat_aliya_names[heb_id[1][0]]]
                versetext_he = r"\aliya{"+aliya_text[1]+r"\newline ("+aliya_text[0]+")}"
            elif heb_id[0] == "aliya":
                aliya_text = shabbat_aliya_names[heb_id[1]]
                versetext_he = r"\aliya{"+aliya_text+"}"
            if versecounter == 1:
                versetext_en = r'\Roman{chap}'
            else:
                versetext_en = r"\arabic{verse}"
            rashi_pasuk = parserashi(rashi,chapcounter,versecounter)
            targ_vs = targ["text"][chapcounter-1][versecounter-1]
            english = parseenglish(eng["text"][chapcounter-1][versecounter-1])
            english_notes = addnotes(book,chapcounter,versecounter)
            if english_notes != None:
                output.append(r"\engnote{"+english_notes+r"}")
            output.append(r'\threeverse{'+versetext_he+"}%Ex."+str(chapcounter)+r':'+str(versecounter))
            if rashi_pasuk != "":
                output.append(r'{'+verse_he)
                output.append(rashi_pasuk+"}")
            else:
                output.append(r'{'+verse_he+'}')
            output.append(r"{"+targ_vs+r"}")
            output.append("{"+english+r"}{"+versetext_en+r"}")
            versecounter += 1
        chapcounter += 1
        output.append(r"\newperek")
        versecounter = 1
    #output[-1] = r"}"
    return output

In [215]:
result = make_chumash(MAPM_ex, targ_ex, JPS_ex, rashi_ex, aliyot, 2)

In [216]:
def TeXwriter(outputfile,content):
    with open(outputfile, 'w', encoding='utf-8') as outfile:
        for line in content:
            outfile.write(line)
            outfile.write("\n")

In [217]:
TeXwriter("exodus_multi.tex",result)